In [1]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-
# scrape_every_hour

import re
import time
import os
import requests
import logging
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from celery_app import app
from fake_useragent import UserAgent
import json
import gspread
from df2gspread import df2gspread as d2g
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from celery.utils.log import get_task_logger

logger = get_task_logger('celery_app')

ua = UserAgent()
fake_header = {
    "user-agent":ua.random
}
params = (
    ('f', 'json'),
    ('where', '1=1'),
    ('returnGeometry', 'false'),
    ('spatialRel', 'esriSpatialRelIntersects'),
    ('outFields', '*'),
    ('orderByFields', 'OBJECTID ASC'),
    ('resultOffset', '0'),
    ('resultRecordCount', '1000'),
    ('cacheHint', 'true'),
    ('quantizationParameters', '{"mode":"edit"}'),
)

def scrape(params):
	try:
		query_url = "https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/ncov_cases/FeatureServer/1/query"
		response = requests.post(query_url,fake_header,params=params)
		return response
	except Exception as e:
		logger.error("下载数据失败")

def parse(response):
	json_files = json.loads(response.text)
	table_every_country = pd.DataFrame(list(map(lambda x:x["attributes"],json_files["features"])))
	table_table_total = table_every_country.groupby("Country_Region").agg({"Last_Update":"first","OBJECTID":"first","Confirmed":"sum",
                                                                   "Recovered":"sum","Deaths":"sum"})
	tf = table_table_total.sort_values(by = "Confirmed",ascending = False)
	countries = pd.read_excel("country_sequence.xlsx")
	tf_f = tf.reindex(list(countries["Country"]))
	return tf_f

def generate_time():
	nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
	return str(nowTime)

def export_to_gs_every_hour(tf_f):
	wks_name_old = "Old_version"
	wks_name_latest = 'Latest_data'
	spreadsheet_key = '1kNgqN0an1xePNfqkXD8bMQDiBWtanAOfiDSNUn22Ln8'

	scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
	credentials = ServiceAccountCredentials.from_json_keyfile_name('jsonFileFromGoogle.json', scope)

	# 第四步：把新的版本上传到latest_version
	# tf ~ latest_version
	try:
		d2g.upload(tf_f, spreadsheet_key, wks_name_latest, credentials=credentials, row_names=True)
	except Exception as e:
		logger.error(e)
		logger.error("导入新版本数据失败！")

	# 第五步：留存最新版本记录(.xlsx)
	nowTime = generate_time()
	filename_new = str(nowTime) + "_new_version.xlsx"
	tf_f.to_excel(filename_new)

def execute():
	logger.info("每小时的任务 - 开始启动")
	response = scrape(params)
	tf_f = parse(response)
	export_to_gs_every_hour(tf_f)

	logging_time = generate_time()
	logging_records = str(logging_time) + "_每小时任务_更新完毕"
	logger.info(logging_records)
    
#execute()

In [21]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-
import re
import time
import os
import requests
import logging
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from celery_app import app
from fake_useragent import UserAgent
import json
import gspread
from df2gspread import df2gspread as d2g
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from celery.utils.log import get_task_logger
import sys
import warnings

warnings.simplefilter("ignore")
logger = get_task_logger('celery_app')

wks_name_old = "Old_version"
wks_name_latest = 'Latest_data'
wks_name_history = "oversea_history_test"
spreadsheet_key = '1kNgqN0an1xePNfqkXD8bMQDiBWtanAOfiDSNUn22Ln8'

def create_cre():
	scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
	credentials = ServiceAccountCredentials.from_json_keyfile_name('jsonFileFromGoogle.json', scope)
	return credentials

def generate_time():
	nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
	return str(nowTime)

def data_transfer(table_data):
    new_header = table_data.iloc[0] #grab the first row for the header
    table_data = table_data[1:] #take the data less the header row
    table_data.columns = new_header #set the header row as the df header
    return table_data

def table_transfer(table,column,day):
    table_t = table[["name","province",column]]
    table["stats_type"] = column
    table = table[["name","province","stats_type",column]]
    table.rename(columns = {column:day},inplace = True)
    return table

def execute():
	logger.info("历史数据表开始更新")
	credentials = create_cre()
	gc = gspread.authorize(credentials)
	spreadsheet = gc.open_by_key(spreadsheet_key)
	table_data = pd.DataFrame(spreadsheet.worksheet("Map_oversea").get_all_values())
	history_data = pd.DataFrame(spreadsheet.worksheet("oversea_history_test").get_all_values())

	table_data_2 = data_transfer(table_data)
	history_data_final = data_transfer(history_data)

	# 得到今天的时间
	day = '{dt.month}月{dt.day}日'.format(dt = datetime.datetime.now())
	table_data_3 = table_data_2.drop(["id"],axis = 1)

	table_casetotal = table_transfer(table_data_3,"casetotal",day)
	table_curecase = table_transfer(table_data_3,"curecase",day)
	table_deathcase = table_transfer(table_data_3,"deathcase",day)

	# 将得到的数据与原表merge得到顺序，然后贴进去
	table_final = pd.concat([table_casetotal,table_curecase,table_deathcase])
	table_history_final = pd.merge(history_data_final,table_final,on = ["name","stats_type"],how = "right")
	history_data_final.loc[:,day] = list(table_history_final[day+"_y"])

	# 存成excel   
	nowTime = generate_time()
	filename = str(nowTime) + "_history_data.xlsx"
	history_data_final.to_excel(filename)

	try:
		d2g.upload(history_data_2, spreadsheet_key, wks_name_history, credentials=credentials, row_names=False)
		logger.info("历史数据表更新完毕")
	except Exception as e:
		logger.error("历史数据更新失败，请检查网络！")
        
execute_2()

In [95]:
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import webbrowser
from oauth2client import client
from oauth2client.file import Storage
import datetime

wks_name_old = "Old_version"
wks_name_latest = 'Latest_data'
spreadsheet_key = '1kNgqN0an1xePNfqkXD8bMQDiBWtanAOfiDSNUn22Ln8'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('jsonFileFromGoogle.json', scope)

gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_key(spreadsheet_key)
table_data = pd.DataFrame(spreadsheet.worksheet("Map_oversea").get_all_values())
history_data = pd.DataFrame(spreadsheet.worksheet("oversea_history").get_all_values())

In [134]:
def data_transfer(table_data):
    new_header = table_data.iloc[0] #grab the first row for the header
    table_data= table_data[1:] #take the data less the header row
    table_data.columns = new_header #set the header row as the df header
    return table_data

table_data_2 = data_transfer(table_data)
history_data_2 = data_transfer(history_data)

In [199]:
day = '{dt.month}月{dt.day}日'.format(dt = datetime.datetime.now())
table_data_3 = table_data_2.drop(["id"],axis = 1)

def table_transfer(table,column):
    table_t = table[["name","province",column]]
    table["stats_type"] = column
    table = table[["name","province","stats_type",column]]
    table.rename(columns = {column:day},inplace = True)
    return table

table_casetotal = table_transfer(table_data_3,"casetotal")
table_curecase = table_transfer(table_data_3,"curecase")
table_deathcase = table_transfer(table_data_3,"deathcase")
table_final = pd.concat([table_casetotal,table_curecase,table_deathcase])
table_history_final = pd.merge(history_data_2,table_final,on = ["name","stats_type"],how = "right")
history_data_2["4月14日"] = list(table_history_final["4月14日_y"])

/Users/dfuser/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [206]:
history_data_2
#.set_index("name").reindex("")

,name,area,stats_type,4月15日,4月14日,4月13日,4月12日,4月11日,4月10日,4月9日,...,1月24日,1月23日,1月22日,1月21日,1月20日,1月19日,1月18日,1月17日,1月16日,1月15日
1,中国,亚洲,casetotal,83696,83696,83597,83482,83386,83305,83250,...,927,638,548,320,218,62,45,41,41,41
2,中国,亚洲,curecase,78262,78262,78145,78020,77935,77838,77711,...,36,31,28,25,25,19,15,12,12,7
3,中国,亚洲,deathcase,3351,3351,3351,3349,3349,3345,3344,...,26,17,17,6,4,2,2,2,2,1
4,泰国,亚洲,casetotal,2613,2613,2579,2551,2518,2473,2423,...,5,4,4,2,2,2,2,2,0,0
5,澳大利亚,大洋洲,casetotal,6400,6494,6359,6313,6292,6152,6052,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,马尔维纳斯群岛,南美洲,deathcase,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
684,圣多美和普林西比,非洲,deathcase,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
685,也门,亚洲,deathcase,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
686,圣皮埃尔岛和密克隆岛,北美洲,deathcase,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
'''
# casetotal
table_casetotal = table_data_3[["name","province","casetotal"]]
table_casetotal["stats_type"] = "casetotal"
table_casetotal = table_casetotal[["name","province","stats_type","casetotal"]]
table_casetotal.rename(columns = {"casetotal":day},inplace = True)

# curecase
table_curecase = table_data_3[["name","province","curecase"]]
table_curecase["stats_type"] = "curecase"
table_curecase = table_curecase[["name","province","stats_type","curecase"]]
table_curecase.rename(columns = {"curecase":day},inplace = True)

# deathcase
table_deathcase = table_data_3[["name","province","deathcase"]]
table_deathcase["stats_type"] = "deathcase"
table_curecase = table_casetotal[["name","province","stats_type","deathcase"]]
'''

'\n# casetotal\ntable_casetotal = table_data_3[["name","province","casetotal"]]\ntable_casetotal["stats_type"] = "casetotal"\ntable_casetotal = table_casetotal[["name","province","stats_type","casetotal"]]\ntable_casetotal.rename(columns = {"casetotal":day},inplace = True)\n\n# curecase\ntable_curecase = table_data_3[["name","province","curecase"]]\ntable_curecase["stats_type"] = "curecase"\ntable_curecase = table_curecase[["name","province","stats_type","curecase"]]\ntable_curecase.rename(columns = {"curecase":day},inplace = True)\n\n#deathcase\ntable_deathcase = table_data_3[["name","province","deathcase"]]\ntable_deathcase["stats_type"] = "deathcase"\ntable_curecase = table_casetotal[["name","province","stats_type","deathcase"]]\n'

In [281]:
history_data_2[day] = list(table_history_final[day+"_y"])

/Users/dfuser/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [279]:
history_data_2.loc[:,day] = pd.DataFrame(range(len(history_data_2))).loc[:,0]

#history_data_2
#list(table_history_final[day+"_y"])
#history_data_2
#history_data_2

In [280]:
wks_name_history = "oversea_history_test"
history_data_2.reset_index(inplace = True)
d2g.upload(history_data_2,spreadsheet_key,wks_name_history,credentials=credentials,row_names=False)

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
'''
def all_update(spreadsheet):
    # 第一步，下载且变换表格
    #table_old_version_data = pd.DataFrame(spreadsheet.worksheet("Latest_data").get_all_values())
    #new_header = table_old_version_data.iloc[0] #grab the first row for the header
    #table_old_version_data = table_old_version_data[1:] #take the data less the header row
    #table_old_version_data.columns = new_header #set the header row as the df header
    #table_old_version_data = table_old_version_data.set_index("")

    # 第二步：保存原始记录（最好也能上传到github）(.xlsx)
    # log全部记录到一个数据库表格里！
    #nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    #filename_old = str(nowTime) + "_old_version.xlsx"
    #table_old_version_data.to_excel(filename_old)

    # 第三步：上传原始记录到old_version
    #d2g.upload(table_old_version_data, spreadsheet_key, wks_name_old, credentials=credentials, row_names=True)

    # 第四步：把新的版本上传到latest_version
    # tf ~ latest_version
    d2g.upload(tf_f, spreadsheet_key, wks_name_latest, credentials=credentials, row_names=True)

    # 第五步：留存最新版本记录(.xlsx)
    nowTime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    filename_new = str(nowTime) + "_new_version.xlsx"
    tf.to_excel(filename_new)

#df_old = g2d.download(spreadsheet_key, wks_name_latest, col_names = True, row_names = False)
#spreadsheet.worksheet("latest_data").get_all_values()
#d2g.upload(df_old, spreadsheet_key, wks_name_old, credentials=credentials, row_names=True)
#d2g.upload(tf, spreadsheet_key, wks_name_latest, credentials=credentials, row_names=True)
'''
'''
1.一定检查港澳台，表格顺序不要更改了！
2.JHU不包含的我们表中45个小地方，首次更新后，每次更新时就再看一看有没有增加就可以。
3.45个小地方均为手动解决问题好了，classification为2，更新频率不大。
绿色区域手动更新
'''
# 头部文件要试验
"""
headers = {
    'authority': 'services1.arcgis.com',
    'cache-control': 'no-cache',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
    'sec-fetch-dest': 'image',
    'accept': 'image/webp,image/apng,image/*,*/*;q=0.8',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-user': '?1',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cookie': '_ga=GA1.2.1425353036.1586234244; _gid=GA1.2.1527009309.1586234244; check=true; AMCVS_ED8D65E655FAC7797F000101%40AdobeOrg=1; _gcl_au=1.1.270179644.1586243707; s_cc=true; sat_track=true; pi_opt_in8202=true; _fbp=fb.1.1586243708663.1796668146; esri_gdpr=true; esri_aopc=-I2ecIeccYaUMrfuBPs8RHcTyBR7XTyuRECDCssS-qaegT2demM_P82T6laWfv-cp3XOWWBTbjdPoKpGDKaIOaBQu1KdByw-EW6qpnWSiFfdyBIckVBiSNk4kY6SLKd9pGDIFcXDXaJfiiuBQiZayVNUJccW9Z7HsLQrOVKB2NdKaYZHZmuIlhdLVkaTWS2X4WkvwH0ICeiVLJxTdBxashe6RN4JAPP16ETQFayPc1ZHVmbIsCJkNuySWx2bIn62YIiWXamopgizZ9TmXCU_yexz8q4jIhbrL9gYRi6ZXMTe4n8cfdqZmBnfoeza5N5UW-60-rcQyWM7MEEgo6Kn6m6s2nsFsXv3kzxdONNysRdm2IQOQhaZAEjSpyE1lQ8xKoqmnlDg8AORx5kXDZDf-8_0M57Sd5d9iIxRC-Qsu1T2WDm_gVxOnyg6UfNL0KErxUjPuevVQcRZCmjBy-rA8IHoPZWiqt-QNss6RJsMAxsZ16IWkcwrdIozlD3DjZLf; esri_auth=%7B%22email%22%3A%22hongruzyj%22%2C%22token%22%3A%223nBiyZCC6Larg5DNbcrPSToCYyLx_shG3dMMfJ7ITKwXT00wfGUNAFRAEEYRzAQ7UGoWxYPjJLBIUSDWylIB1BnGxGK-4mlL5IWjmi7cFuXNi_EkO6pPoW3jl25kqOrq0eMzPeTNzP36q9p9hAGAQzHo4tGm_Os_ubi6rj1UyJ2N7jGjAPD0Pa8rKkpqanla%22%2C%22culture%22%3A%22%22%2C%22expires%22%3A1587453955079%2C%22created%22%3A1586244355079%2C%22persistent%22%3Afalse%2C%22id%22%3A%22xa9tM201ZkXgvcDP%22%2C%22customBaseUrl%22%3A%22www.arcgis.com%22%7D; s_dslv_s=First%20Visit; AAMC_esri_0=REGION%7C9; gpv_v9=developers.arcgis.com%3A%20labs%3A%20rest%3A%20query-a-feature-layer; s_ptc=%5B%5BB%5D%5D; s_sq=%5B%5BB%5D%5D; s_tp=5264; dmdbase_cdc=DBSET; mbox=session#166312b3464347f9a0e48e031d89847a#1586247387|PC#166312b3464347f9a0e48e031d89847a.28_0#1649489273; AMCV_ED8D65E655FAC7797F000101%40AdobeOrg=-1303530583%7CMCIDTS%7C18360%7CMCMID%7C81287045279634030932150418578290471367%7CMCAAMLH-1586850621%7C9%7CMCAAMB-1586850621%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1586253021s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C3.3.0%7CMCCIDH%7C-97020212; s_dslv=1586245821861; gpv_pn=developers.arcgis.com%3A%20javascript%3A%20latest%3A%20api-reference%3A%20esri-tasks-support-query.html; tp=25767; s_ppv=developers.arcgis.com%253A%2520javascript%253A%2520latest%253A%2520api-reference%253A%2520esri-tasks-support-query.html%2C87%2C5%2C22394',
    'if-none-match': 'sd86582_-1274114320',
    'if-modified-since': 'Tue, 07 Apr 2020 06:56:03 GMT',
    'pragma': 'no-cache',
    'referer': 'https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/ncov_cases/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=1000&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D',
}
"""